## config

In [1]:
import os
import glob
import re
import pandas as pd

In [2]:
## config the path where multiple files contained.
## NEED modify every time it runs !!!

path = "/home/jovyan/work/10.data_CODA_ahslyy/0014215947_CA/results/09.EpitopePrediction"
homobed_or_blasp = "homoblastp"
'''
select list is "homobed","homoblastp","bedblastp"
"homobed" means homobed file as input only
"homoblastp" means homoblastp file as input only
"bedblastp" means homobed and homoblastp files both as input
'''
identity = 75

In [3]:
## config input files under the given path.
## No NEED modify every time it runs !!!

path_mhcI = path + "/02.protein_antigen_prediction/parsed_res_MHCI"
path_mhcII = path + "/02.protein_antigen_prediction/parsed_res_MHCII"
trans_memb = path + "/04.TransMembrane.DeepTMHMM/TransMembrane.bed"
enzymedigest = path + "/05.EnzymeDigest/protein.merge.enzymedigest"

if homobed_or_blasp=="homobed":
    homo_bed = path + "/03.homologous/homo_peptide.bed"
elif homobed_or_blasp== "homoblastp":
    homo_blastp = path + "/03.homologous/peptide.netMHC.aa.blastp"
elif homobed_or_blasp== "bedblastp":
    homo_bed = path + "/03.homologous/homo_peptide.bed"
    homo_blastp = path + "/03.homologous/peptide.netMHC.aa.blastp"

outpath = path + "/06.Deliverables/EpitopePresent"
outpath1 = path + "/06.Deliverables/EpitopePresent/MHCI"
outpath2 = path + "/06.Deliverables/EpitopePresent/MHCII"
try:
    os.mkdir(outpath)
except:
    pass
try:
    os.mkdir(outpath1)
except:
    pass
try:
    os.mkdir(outpath2)
except:
    pass

#### Tools

In [4]:
import difflib
def get_overlap(s1, s2):
    s = difflib.SequenceMatcher(None, s1, s2)
    pos_a, pos_b, size = s.find_longest_match(0, len(s1), 0, len(s2)) 
    return(size,s1[pos_a:pos_a+size])
s1 = "CGATTCCAGGCTCCCCACGGGGTACCCATAACTTGACAGTAGATCTC"
s2 = "GGCTCCCCACGGGGTACCCATAACTTGACAGTAGATCTCGTCCAGACCCCTAGC"

print(get_overlap(s1, s2)) # GGCTCCCCACGGGGTACCCATAACTTGACAGTAGATCTC

(39, 'GGCTCCCCACGGGGTACCCATAACTTGACAGTAGATCTC')


In [5]:
def get_homodict_bed(Homo_parsDict,seqid,pos1,pos2):
    checklist = [homo_f==0 for homo_f in Homo_parsDict[seqid][pos1:pos2]]
    if  sum(checklist)/(pos2-pos1) >= 0.05:
        HomoEx = "N"
        Homo_flag = ""
    else:
        HomoEx = "Y"
        flagtemp_list = []
        for flag in Homo_parsDict[seqid][pos1:pos2]:
            if flag !=0:
                flagtemp_list += flag
        Homo_flag = ";".join(list(dict.fromkeys(flagtemp_list)))
    return(HomoEx,Homo_flag)

def get_homodict_blastp(Homoblastp_parsDict,seqid,peptide):
    try:
        homolist = Homoblastp_parsDict[seqid][peptide]
        if len(homolist) == 0:
            Homo_flag = ""
            HomoEx = "N"
        else:
            Homo_flag = ';'.join(homolist)
            HomoEx = "Y"
    except:
        Homo_flag = ""
        HomoEx = "N"
    return(HomoEx,Homo_flag)
    

## Step1: Parser the TransMemb.bed

In [6]:
## Parser the TransMemb.bed
TM_parsDict={}
flagDict={"o":"outside","T":"TMhelix","i":"inside","s":"signal"}
'''
    parser the tranmembe.bed file
    TM_flag=outside,TMhelix,inside
    outside:o
    TMhelix:t
    inside:i
'''
with open(trans_memb,"r") as tmf:
    for line in tmf.readlines():
        ### 对文件第一行跳过
        if line.startswith("track name="):
            continue 
        ### 对用Sakit分析得到的蛋白，获取蛋白ID
        elif line.startswith("PB"):
            seqid = line.split("\t")[0]
            seqid = seqid.replace(".","_")
            seqid = "_".join(seqid.split("_")[0:3])
        ### 对根据protein ID分析得到的蛋白，获取蛋白ID
        elif line.startswith("sp"):
            seqid = line.split("\t")[0]
            seqid = seqid.replace("|","_")
            seqid = "_".join(seqid.split("_")[0:2])
            
        print(seqid)
        ### 
        if seqid not in TM_parsDict.keys():
            TM_parsDict[seqid] = "" ### 初始化 seqid在TM_parsDict的存储
        pos1 = int(line.split("\t")[1])
        pos2 = int(line.split("\t")[2])
        TM_flag = line.split("\t")[3][0]
        TM_parsDict[seqid] += TM_flag*(1+pos2-pos1)
print(TM_parsDict)

PB_151_COPA
PB_1595COL10A1
PB_1595COL10A1
{'PB_151_COPA': 'iiiiiiiiiiiiiiiiiiiiiiii', 'PB_1595COL10A1': 'ssssssssssssssssssoooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooo'}


## Step2: Parser the Homologous_peptide.bed

In [7]:
## Parser the Homologous_peptide.bed
if (homobed_or_blasp == "homobed") or (homobed_or_blasp == "bedblastp"):
    Homobed_parsDict={}
    with open(homo_bed,"r") as hmf:
        for line in hmf.readlines():
            if line.startswith("sp|"):
                #print(line)
                seqid = line.split("\t")[0].split("|")[1]
                if seqid not in Homobed_parsDict.keys():
                    length = len(TM_parsDict[seqid])
                    Homobed_parsDict[seqid] = [0]*length
                pos1 = int(line.split("\t")[1])-1
                pos2 = int(line.split("\t")[2])-1
                homoid = line.split("\t")[3]
                for pos in range(pos1,pos2):
                    #print(pos)
                    if Homobed_parsDict[seqid][pos] == 0:
                        Homobed_parsDict[seqid][pos] = [homoid]
                    else:
                        Homobed_parsDict[seqid][pos].append(homoid)

In [8]:
## Parser the peptide_netMHC.blastp
if (homobed_or_blasp == "homoblastp") or (homobed_or_blasp == "bedblastp"):
    Homoblastp_parsDict={}
    headname= ["qseqid","sseqid","pident","length","mismatch","gapopen","qstart","qend","sstart","send","evalue","bitscore"]
    blastdf = pd.read_csv(homo_blastp,sep="\t",names=headname)
    blastdf[["protein","peptide"]] = blastdf.qseqid.str.split(":",expand=True)
    #print(blastdf.head(10))
    blastdf["proteinUnipID"] = blastdf["protein"].str.split("_").apply(lambda x: "_".join(x[0:3]))
    print(blastdf.proteinUnipID.unique().tolist())
    proteinlist = blastdf.proteinUnipID.unique().tolist()
    selfUniproList = []
    for proteinid in proteinlist:
        # calculate the self 
        qblastdf = blastdf[blastdf["proteinUnipID"]==proteinid]
        homoRefseries = qblastdf.sseqid.value_counts()
        # calculate the query number (or bin numbers)
        querynum = len(qblastdf.qseqid.unique())
        # find those reference id which is the query ploymer 
        selfUniproID = homoRefseries[homoRefseries>querynum*0.9].index.tolist()
        selfUniproList += selfUniproID
    print(selfUniproList)
 
    # remove those aligned lines which reference is it's self
    noSelfBlastdf = blastdf[~blastdf.sseqid.isin(selfUniproList)]
    # select those aligned lines which pident bigger than cutoff
    noSelfHomodf = noSelfBlastdf[noSelfBlastdf.pident > identity] # identity=65
    
    noSelfHomodf2 = noSelfHomodf[["proteinUnipID","peptide","sseqid"]]
    #print(noSelfHomodf2.head(20))
    for proteinid in proteinlist:
        print(proteinid)
        noSelfHomodf2_p = noSelfHomodf2[noSelfHomodf2["proteinUnipID"]==proteinid]
        peptidelist = noSelfHomodf2_p.peptide.unique().tolist()
        peptidesseqiddict = {}
        for peptide in peptidelist:
            sseqidlist = noSelfHomodf2_p[noSelfHomodf2_p["peptide"]==peptide].sseqid.tolist()
            peptidesseqiddict[peptide] = sseqidlist
            Homoblastp_parsDict[proteinid] = peptidesseqiddict
    #print(Homoblastp_parsDict)
  

['PB_151_COPA', 'PB_1595COL10A1']
['Q8CIE6']
PB_151_COPA
PB_1595COL10A1


In [9]:
#print(Homoblastp_parsDict["Q96A99"])

## Step3: Parser the enzymedigest file

In [10]:
## Parser the enzymedigest file
##适用于TAA的分析
EnzDigeDict={}
with open(enzymedigest,"r") as enzymef:
    for line in enzymef.readlines():
        print(line)
        if (''',"[''' in line) and (''']"''' in line):
            #print(line)
            refseq = re.split(',"\[',line)[0].split(",")[-1]
            #print(refseq)
            refid = line.split(",")[0].split("_")[0].replace(".","_").replace("|","_").replace(">","")
            seqlist = re.split(',"\[',line)[1].strip().replace(']"','').replace("'","").split(",")
        elif (''',[''' in line) and (''']''' in line):
            #print(line)
            refseq = re.split(',\[',line)[0].split(",")[-1]
            print(refseq)
            refid = line.split(",")[0].split("_")[0].replace(".","_").replace("|","_").replace(">","")
            seqlist = re.split(',\[',line)[1].strip().replace(']"','').replace("'","").split(",")
        EnzDigeDict[refid]=seqlist
print(EnzDigeDict)

{}


In [11]:
## Parser the netMHC.csv
mergeoutfile =  open(outpath+"/MHCI_Epitopeonly_SB_WB.csv","w")
filenum = 0
for file in glob.glob(path_mhcI + "/*.netMHC.csv"):
    filenum +=1
    filename= os.path.basename(file)
    print(filename)
    outfile = open(outpath1+"/"+filename,"w")
    with open(file,"r") as eptf:
        for line in eptf.readlines():
            itemlist = line.rstrip().split(",")
            """
            itemlist is the line content of netMHC result
            """
            if line.startswith("Pos"):
                appendlist = ["TransMemb","InCutmerRate","InCutmerRegion","HomoExsit","HomoId"]
                outfile.write(",".join(itemlist+appendlist)+"\n")
                if filenum == 1:
                    mergeoutfile.write(",".join(itemlist+appendlist)+"\n")
            else:
                pos1 = int(itemlist[0])-1
                peptide = itemlist[2]
                pos2 = pos1 + len(peptide)
                midpos = int((pos1+pos2)/2)

                seqid =  "_".join(itemlist[10].split("_")[0:3]) 
                '''
                if itemlist[10][-1] != '_':
                    seqid =  "_".join(itemlist[10].split("_")[0:3]) 
                else:
                    seqid = itemlist[10][:len(itemlist[10])-1]
                '''
                
                #print(seqid,midpos)
                ## get Transmembrane info
                TM_f = TM_parsDict[seqid][midpos]
                TM_flag = flagDict[TM_f]
                
                ## get enzymedigest info
                Cut_flag = "_"
                Cut_seq = "_"
                '''
                size0 = 0
                overseq0 = ""
                for seq in EnzDigeDict[seqid]:
                    size,overseq = get_overlap(peptide,seq)
                    if size > size0:
                        size0 = size
                        overseq0 = overseq
                Cut_flag = '"'+str(size0)+"//"+str(len(peptide))+'"'
                Cut_seq = overseq0
                '''
                
                ## get homolgous info
                if (homobed_or_blasp == "homobed") or (homobed_or_blasp == "bedblastp"):
                    HomoEx,Homo_flag = get_homodict_bed(Homo_parsDict,seqid,pos1,pos2)
                elif (homobed_or_blasp == "homoblastp") or (homobed_or_blasp == "bedblastp"):
                    HomoEx,Homo_flag = get_homodict_blastp(Homoblastp_parsDict,seqid,peptide)
                ##  output the results with filtering the "SB" lines
                if (itemlist[-1] == "SB") or (itemlist[-1] == "WB"):
                    print(itemlist[-1])
                    appendlist = [TM_flag,Cut_flag,Cut_seq,HomoEx,Homo_flag]
                    mergeoutfile.write(",".join(itemlist+appendlist)+"\n")
                else:
                    appendlist = ["",TM_flag,Cut_flag,HomoEx,Homo_flag]
                outfile.write(",".join(itemlist+appendlist)+"\n")
    outfile.close()

PB_1595COL10A1_MHCI_HLA-B*15:07.netMHC.csv
SB
SB
SB
SB
SB
SB
SB
SB
SB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
PB_151_COPA_MHCI_HLA-B*15:07.netMHC.csv
SB
WB
PB_151_COPA_MHCI_HLA-C*03:03.netMHC.csv
PB_1595COL10A1_MHCI_HLA-C*03:03.netMHC.csv
SB
SB
SB
SB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
PB_151_COPA_MHCI_HLA-B*67:01.netMHC.csv
WB
PB_151_COPA_MHCI_HLA-C*07:02.netMHC.csv
WB
PB_151_COPA_MHCI_HLA-A*24:02.netMHC.csv
PB_1595COL10A1_MHCI_HLA-B*67:01.netMHC.csv
SB
SB
SB
SB
SB
SB
SB
SB
SB
SB
SB
SB
SB
SB
SB
SB
SB
SB
SB
SB
SB
SB
SB
SB
SB
SB
SB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
WB
PB_151_COPA_MHCI_HLA-A*02:01.netMHC.csv
PB_1595COL10A1_MHCI_HLA-C*07:02.netMHC.csv
SB
SB
SB
SB
SB
SB
SB
SB
SB
SB
SB
WB
WB
WB
WB
WB
WB
WB
WB
WB


In [12]:
## Parser the netMHC.csv
mergeoutfile2 =  open(outpath+"/MHCII_Epitope_only_SB_WB.csv","w")
filenum = 0
for file in glob.glob(path_mhcII + "/*.netMHC.csv"):
    filenum += 1
    filename= os.path.basename(file)
    print(filename)
    outfile = open(outpath2+"/"+filename,"w")
    with open(file,"r") as eptf:
        for line in eptf.readlines():
            itemlist = line.rstrip().split(",")
            if line.startswith("Pos"):
                appendlist = ["TransMemb","InCutmerRate","InCutmerRegion","HomoExsit","HomoId"]
                outfile.write(",".join(itemlist+appendlist)+"\n")
                if filenum == 1:
                    mergeoutfile2.write(",".join(itemlist+appendlist)+"\n")
            else:
                pos1 = int(itemlist[0])-1
                peptide = itemlist[2]
                pos2 = pos1 + len(peptide)
                midpos = int((pos1+pos2)/2)

                seqid = "_".join(itemlist[6].split("_")[0:3])
                '''
                if itemlist[6][-1] != '_':
                    seqid = itemlist[6]  
                else:
                    seqid = itemlist[6][:len(itemlist[6])-1]
                seqid = seqid.replace("|","_").replace(".","_")
                '''
                
                ## get Transmembrane info
                TM_f = TM_parsDict[seqid][midpos]
                TM_flag = flagDict[TM_f]
                
                ## get enzymedigest info
                Cut_flag = "_"
                '''
                size0 = 0
                overseq0 = ""
                for seq in EnzDigeDict[seqid]:
                    size,overseq = get_overlap(peptide,seq)
                    if size > size0:
                        size0 = size
                        overseq0 = overseq
                Cut_flag = '"'+str(size0)+"//"+str(len(peptide))+'"'
                Cut_seq = overseq0
                '''
                ## get homolgous info
                if (homobed_or_blasp == "homobed") or (homobed_or_blasp == "bedblastp"):
                    HomoEx,Homo_flag = get_homodict_bed(Homo_parsDict,seqid,pos1,pos2)
                elif (homobed_or_blasp == "homoblastp") or (homobed_or_blasp == "bedblastp"):
                    HomoEx,Homo_flag = get_homodict_blastp(Homoblastp_parsDict,seqid,peptide)
                        
                ##  output the results with filtering the "SB" lines
                if (itemlist[-1] == "SB") or (itemlist[-1] == "WB"):
                    appendlist = [TM_flag,Cut_flag,Cut_seq,HomoEx,Homo_flag]
                    mergeoutfile2.write(",".join(itemlist+appendlist)+"\n")
                else:
                    appendlist = ["",TM_flag,HomoEx,Homo_flag]
                outfile.write(",".join(itemlist+appendlist)+"\n")
    outfile.close()

PB_151_COPA_MHCII_DRB5_0202.netMHC.csv
PB_1595COL10A1_MHCII_DRB4_0103.netMHC.csv
PB_1595COL10A1_MHCII_DRB5_0202.netMHC.csv
PB_151_COPA_MHCII_DRB4_0103.netMHC.csv
PB_151_COPA_MHCII_DRB1_0403.netMHC.csv
PB_1595COL10A1_MHCII_DRB1_0403.netMHC.csv
PB_1595COL10A1_MHCII_DRB1_1602.netMHC.csv
PB_151_COPA_MHCII_DRB1_1602.netMHC.csv


In [13]:
df_SB_WB = pd.read_csv(outpath+"/MHCI_Epitopeonly_SB_WB.csv")
proteinlist = list(df_SB_WB["Identity"].unique())
for seqid in proteinlist:
    print(seqid)
    try:
        os.mkdir(outpath+"/"+seqid)
    except:
        pass
    seqid_df = df_SB_WB[df_SB_WB["Identity"].str.contains(seqid)]
    MHClist= (seqid_df["MHC"].unique())
    for MHCid in MHClist:
        print(MHCid)
        seqid_mhcid_df = seqid_df[seqid_df["MHC"].str.contains(MHCid,regex=False)]
        seqid_mhcid_df.to_csv(outpath+"/"+seqid+"/"+seqid+"_MHCI_"+MHCid+".netMHC.csv", index=False)
        

PB_1595COL10A1
HLA-B*15:07
HLA-C*03:03
HLA-B*67:01
HLA-C*07:02
HLA-A*02:01
PB_151_COPA
HLA-B*15:07
HLA-B*67:01
HLA-C*07:02


In [14]:
df_SB_WB = pd.read_csv(outpath+"/MHCII_Epitope_only_SB_WB.csv")
proteinlist = list(df_SB_WB["Identity"].unique())
for seqid in proteinlist:
    seqid2 = seqid.replace("|","_").replace(".","_")
    try:
        os.mkdir(outpath+"/"+seqid2)
    except:
        pass
    seqid_df = df_SB_WB[df_SB_WB["Identity"].str.contains(seqid)]
    MHClist= (seqid_df["MHC"].unique())
    for MHCid in MHClist:
        seqid_mhcid_df = seqid_df[seqid_df["MHC"].str.contains(MHCid)]
        seqid_mhcid_df.to_csv(outpath+"/"+seqid2+"/"+seqid2+"_MHCII_"+MHCid+".netMHC.csv", index=False)